<a href="https://colab.research.google.com/github/yandod/machine-learning-examples/blob/main/08_openai_completion_and_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* このノートはColabのシークレット内にOpenAIのAPIキーが設定されていることを前提としています
* OPENAI_KEY というシークレットを作成し、[有効なAPIキー](https://platform.openai.com/settings/profile?tab=api-keys)を設定してください





In [42]:
#@title 1.スプレッドシートからデータを読み込む
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet_key = '1qTZdjLmSrL8C2AFL6ap9bTMJrWo6iO3D_tv8AQAGqsg' #@param {type:"string"}

#最初のシートを指定
worksheet = gc.open_by_key(worksheet_key).sheet1

data = worksheet.get_all_values()
headers = data[0]  # 一行目をヘッダーとして取得
rows = data[1:]  # 二行目以降をデータとして取得

# データフレームに変換し、一行目をインデックスとして設定
df = pd.DataFrame(rows, columns=headers).set_index(headers[0])
df

,質問,模範解答,理由
id,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地


In [43]:
#@title 2.OpenAI ライブラリのインストール
!pip install openai

from openai import OpenAI
from google.colab import userdata

# OpenAI APIキーを設定
client = OpenAI(api_key=userdata.get('OPENAI_KEY'))

In [44]:
#@title 3.OpenAIのAPIで返答を得る

#モデルの選択
model_name = "gpt-3.5-turbo" #@param ['gpt-3.5-turbo','gpt-4-turbo','gpt-4o']

def get_openai_response(prompt):
  completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
      {
          "role": "user",
          "content": prompt,
      },
  ],
  )
  return completion.choices[0].message.content

# データフレームに新しい列を追加して、各質問に対するOpenAIの出力を保存
df['response'] = df['質問'].apply(get_openai_response)

# データフレームの表示
df

,質問,模範解答,理由,response
id,,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか,晩秋の公園、落ち葉を踏みしめる音だけが響く。ベンチに佇むネコ、琥珀色の瞳が灯る。見つめてくる...
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから,とうきょうとっきょきょかきょく
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地,清泉女子大学は日本の東京都文京区にある女子大学です。


In [45]:
#@title 4.OpenAIのAPIで返答と模範解答を比べて採点する

#モデルの選択
model_name = "gpt-4o" #@param ['gpt-3.5-turbo','gpt-4-turbo','gpt-4o']

def get_openai_grade(row):

  prompt = f"""
  下記の出力が質問に沿っているかを模範解答も参考に１から5で採点してください。
  評価には理由も考慮してください。
  質問: {row['質問']}
  出力: {row['response']}
  模範解答: {row['模範解答']}
  評価基準: {row['理由']}
  """
  completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
      {
          "role": "user",
          "content": prompt,
      },
  ],
  )
  return completion.choices[0].message.content

# データフレームに新しい列を追加して、各質問に対するOpenAIの出力を保存
df['grade'] = df.apply(get_openai_grade, axis=1)

# データフレームの表示
df

,質問,模範解答,理由,response,grade
id,,,,,
1,ネコが出てくる短い話を140文字以内で作ってください,むかしむかし、あるところにとてもおりこうなネコがいました。おりこうなネコはみんなを幸せにしま...,ネコが主要な登場人物かどうか、全体で140文字に収まっているかどうか,晩秋の公園、落ち葉を踏みしめる音だけが響く。ベンチに佇むネコ、琥珀色の瞳が灯る。見つめてくる...,5\n\n理由: 出力は質問の要求を満たしています。ネコは話の中で重要な役割を果たしており、...
2,次の言葉をひらがなに直してください\n東京特許許可局,とうきょうとっきょきょかきょく,正確にひらがなに直っているから,とうきょうとっきょきょかきょく,5
3,清泉女子大学はどこにある,清泉女子大学は、東京都品川区東五反田三丁目16番21号に本部を置く日本の私立大学,品川区東五反田三丁目16番21号が正しい所在地,清泉女子大学は日本の東京都文京区にある女子大学です。,2\n\n出力は日本の女子大学である清泉女子大学が東京都文京区にあると答えていますが、模範解...


In [35]:
#@title 5.処理結果をCSVで保存してダウンロード
from google.colab import files
import csv
df.to_csv('data.csv', quoting=csv.QUOTE_ALL)
files.download('data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>